In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from leap3d.contour import get_melting_pool_contour_2d, get_melting_pool_contour_3d
from leap3d.scanning.scan_parameters import ScanParameters
from leap3d.scanning.scan_results import ScanResults
from leap3d.config import DATA_DIR, PARAMS_FILEPATH, ROUGH_COORDS_FILEPATH, MELTING_POINT
from leap3d.plotting import plot_2d_contours, plot_fake_3d_contours, plot_3d_contours_vertices, plot_3d_contours_faces, plot_cross_section_along_laser_at_timestep

In [ ]:
precompute_path = Path("../data/")

t_start = 2000
steps = 1000
steps_per_file = 100


In [ ]:
def get_data(case_id, t_start, timestep, steps_per_file):
    t = (timestep - t_start) // steps_per_file
    interpolated_3d_data = np.load(precompute_path / f"case_{case_id:04d}_interpolated_3d_data_{t:04d}.npy")
    laser_coordinates = np.load(precompute_path / f"case_{case_id:04d}_laser_coordinates_{t:04d}.npy")

    return interpolated_3d_data, laser_coordinates, t

In [ ]:
current_data_index = -1
loaded_case_id = -1

In [ ]:
case_id = 0
# timestemps: 2000 onwards

# case_id = 1
# timesteps: 2165

case_id = 0
# timesteps: 2000 - 2016
current_timestep = 2000
show_n_top_layers = 24

In [ ]:
if loaded_case_id != case_id:
    loaded_case_id = case_id
    scan_results = ScanResults(DATA_DIR / f"case_{case_id:04d}.npz")
    scan_parameters = ScanParameters(PARAMS_FILEPATH, ROUGH_COORDS_FILEPATH, case_id)

In [ ]:
t = current_timestep - t_start
if t // steps_per_file != current_data_index:
    interpolated_3d_data, laser_coordinates_list, current_data_index = get_data(case_id, t_start, current_timestep, steps_per_file)
t = t % steps_per_file

In [ ]:
temperature = interpolated_3d_data[t]
laser_coordinates = laser_coordinates_list[t]

In [ ]:
contours_2d = get_melting_pool_contour_2d(temperature, top_k=show_n_top_layers)
vertices_3d, faces_3d = get_melting_pool_contour_3d(temperature, top_k=show_n_top_layers)

In [ ]:
temperature_high_res = scan_results.get_melt_pool_temperature_grid(timestep=current_timestep)
vertices_hr_3d, faces_hr_3d = get_melting_pool_contour_3d(temperature_high_res, top_k=show_n_top_layers)

In [ ]:
fig = plt.figure(figsize=(20, 20))
axes = []
axes.append(fig.add_subplot(3, 3, 1))
axes.append(fig.add_subplot(3, 3, 2))
axes.append(fig.add_subplot(3, 3, 3, projection='3d'))
axes.append(fig.add_subplot(3, 3, 4, projection='3d'))
axes.append(fig.add_subplot(3, 3, 5, projection='3d'))

ax = axes[0]
ax.set_aspect('equal')
ax.set_title("Top view")
im = ax.imshow(temperature[:, :, -1], vmin=300, vmax=1500, cmap='hot')

_ = plot_2d_contours(contours_2d[0], ax)

ax = axes[1]
ax.set_title("Cross section")
# ax.set_ylim(64 - show_n_top_layers, 64)
plot_cross_section_along_laser_at_timestep(ax, scan_results, scan_parameters, current_timestep)
# (new_x_grid, new_z_grid, T_values) = cross_sections[0]
# new_z_grid = ((new_z_grid + 0.0006) / 0.0006) * 64
# im = ax.pcolormesh(new_x_grid, new_z_grid, T_values, vmin=300, vmax=1500, cmap='hot')

ax = axes[2]
ax.set_title("Layered 2D contours")
_ = plot_fake_3d_contours(contours_2d, max_n_layers=temperature.shape[2], ax=ax)

ax = axes[3]
ax.set_title("3D contour")
ax.set_aspect('equal')
offset = 10
ax.set_xlim(laser_coordinates[0] - offset, laser_coordinates[0] + offset)
ax.set_ylim(laser_coordinates[1] - offset, laser_coordinates[1] + offset)
ax.set_zlim(64 - show_n_top_layers, 64)
if len(vertices_3d) > 0:
    plot_3d_contours_faces(vertices_3d, faces_3d, ax)[0]

ax = axes[4]
ax.set_title("3D contour high res only")
ax.set_aspect('equal')
offset = 10
ax.set_xlim(laser_coordinates[0] - offset, laser_coordinates[0] + offset)
ax.set_ylim(laser_coordinates[1] - offset, laser_coordinates[1] + offset)
ax.set_zlim(64 - show_n_top_layers, 64)
if len(vertices_hr_3d) > 0:
    plot_3d_contours_faces(vertices_hr_3d, faces_hr_3d, ax)[0]

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
im = ax.imshow(temperature[:, :, -1], vmin=300, vmax=1500, cmap='hot')

_ = plot_2d_contours(contours_2d[0], ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
plot_cross_section_along_laser_at_timestep(ax, scan_results, scan_parameters, current_timestep)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
_ = plot_fake_3d_contours(contours_2d, max_n_layers=temperature.shape[2], ax=ax)

In [ ]:
fig = plt.figure()
ax.set_aspect('equal')
ax = fig.add_subplot(111, projection='3d')
offset = 10
ax.set_xlim(laser_coordinates[0] - offset, laser_coordinates[0] + offset)
ax.set_ylim(laser_coordinates[1] - offset, laser_coordinates[1] + offset)
ax.set_zlim(64 - show_n_top_layers, 64)
plot_3d_contours_faces(vertices_3d, faces_3d, ax)[0]

In [ ]:
rescale = lambda x: (x / 256) * 2 * 0.0006
x_offset = -10
y_offset = -10
gt_laser_x, gt_laser_y = scan_results.get_laser_coordinates_at_timestep(current_timestep)
print(gt_laser_x, gt_laser_y)
new_laser_position = gt_laser_x + rescale(x_offset), gt_laser_y + rescale(y_offset)
print(new_laser_position)

In [ ]:
fig, ax = plt.subplots()
plot_cross_section_along_laser_at_timestep(ax, scan_results, scan_parameters, current_timestep, use_laser_position=new_laser_position)